In [ ]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout
from keras.models import Model
from matplotlib import pyplot as plt
from IPython.display import clear_output

## Data

In [ ]:
features_train = np.load('./features/lstm/encoded/features_train.npy')
features_dev = np.load('./features/lstm/encoded/features_dev.npy')
classes_train = np.load('./features/lstm/encoded/classes_train.npy')
classes_dev = np.load('./features/lstm/encoded/classes_dev.npy')

In [ ]:
print(features_train.shape)
print(features_dev.shape)
print(classes_train.shape)
print(classes_dev.shape)

In [ ]:
input_shape = features_train.shape[1:]
output_shape = classes_train.shape[1]

## Callbacks

In [ ]:
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        f, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=False)
        self.ax1 = ax1
        self.ax2 = ax2
    
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.acc.append(logs.get('categorical_accuracy'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        self.i += 1
        
        clear_output(wait=True)
        self.ax1.plot(self.x, self.losses, label="loss")
        self.ax1.plot(self.x, self.val_losses, label="val loss")
        self.ax1.legend()
        self.ax2.plot(self.x, self.acc, label="accuracy")
        self.ax2.plot(self.x, self.val_acc, label="val accuracy")
        self.ax2.legend()
        plt.show()
        
        for i in range(self.i):
            print('Epoch ' + str(i+1))
            print('-----------------------')
            print('- Loss:', self.losses[i])
            print('- Accuracy:', self.acc[i])
            print('- Validation loss:', self.val_losses[i])
            print('- Validation accuracy:', self.val_acc[i])
            print(' ')
            
checkpoint = keras.callbacks.ModelCheckpoint(
    './convnet_weights/lstm/weights.{epoch:02d}-{val_loss:.2f}.hdf5', 
    monitor='val_loss',
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='max', 
    period=1,
)

## LSTM

In [ ]:
inputs = Input(shape=input_shape)
X = BatchNormalization()(inputs)
X = LSTM(1024)(X)
outputs = Dense(output_shape, activation='softmax')(X)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

## Dense

In [ ]:
features_train_dense = np.concatenate([features_train[:, 0, :], features_train[:, 1, :]], axis=1)
features_dev_dense = np.concatenate([features_dev[:, 0, :], features_dev[:, 1, :]], axis=1)

In [ ]:
inputs = Input(shape=[features_train_dense.shape[1]])
X = BatchNormalization()(inputs)
X = Dense(2048, activation='relu')(X)
outputs = Dense(output_shape, activation='softmax')(X)

model = Model(inputs=inputs, outputs=outputs)
model.summary()

## Training

In [ ]:
plot_losses = PlotLosses()

In [ ]:
model.compile(
    optimizer=tf.train.AdamOptimizer(0.000001),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

model.load_weights('./convnet_weights/lstm/weights.37-1.27.hdf5')

## Train lstm

In [ ]:
model.fit(
    features_train,
    classes_train,
    batch_size=128,
    epochs=100,
    verbose=True,
    callbacks=[plot_losses, checkpoint],
    validation_data=(features_dev, classes_dev)
)

## Train dense

In [ ]:
model.fit(
    features_train_dense,
    classes_train,
    batch_size=128,
    epochs=100,
    verbose=True,
    callbacks=[plot_losses, checkpoint],
    validation_data=(features_dev_dense, classes_dev)
)